In [ ]:
import numpy as np 
import pandas as pd
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import MaxPooling1D, Dense, LeakyReLU, Conv1D 
from tensorflow.keras.layers import Flatten, Activation, BatchNormalization, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time
import pickle

In [ ]:
import tensorflow as tf

In [ ]:
%%time
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]
train.fillna(train.mean(),inplace=True)

In [ ]:
SEED = 1111

tf.random.set_seed(SEED)
np.random.seed(SEED)

train['action'] = ((train['resp'].values) > 0).astype(int)


features = [c for c in train.columns if "feature" in c]

f_mean = np.mean(train[features[1:]].values,axis=0)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

X = train.loc[:, train.columns.str.contains('feature')]
#y_train = (train.loc[:, 'action'])

Y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T


leaky_relu_alpha =0.05
LeakyReLU(alpha=leaky_relu_alpha)


In [ ]:
x_train = X.values.reshape(-1, X.shape[1], 1)  # reshaping for convnet
# x_test = x_test.values.reshape(-1, x_test.shape[1], 1)  # reshaping for convnet

In [ ]:
x_train.shape[1:]

In [ ]:
def build_model():  # random search passes this hyperparameter() object 
    model = keras.models.Sequential()
    model.add(Conv1D(180, 2, input_shape=x_train.shape[1:]))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=leaky_relu_alpha))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.15))  
    model.add(Flatten()) 
    
    model.add(Dense(180))
    model.add(LeakyReLU(alpha=leaky_relu_alpha))
    model.add(Dropout(0.15))  
    model.add(Dense(5))
    model.add(Activation("sigmoid"))

    model.compile(optimizer=keras.optimizers.Adam(lr=1e-3),
                  loss=BinaryCrossentropy(label_smoothing=0.095),
                  metrics=[tf.keras.metrics.AUC(name = 'auc'), "accuracy"])

    return model

In [ ]:
model = build_model()

In [ ]:
model.fit(x=x_train,
          y=Y,
          epochs=10,
          batch_size=1024)

In [ ]:
from tqdm import tqdm

In [ ]:
f = np.median
th = 0.5000
import janestreet
env = janestreet.make_env()
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = np.mean([model(x_tt.reshape(-1, x_tt.shape[1], 1), training = False).numpy()],axis=0)
        pred = f(pred)
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)